In [1]:
import urllib
from selenium.webdriver.common.keys import Keys
# 이게 그 기능을 갖고 있음
from selenium.webdriver.support.ui import WebDriverWait
# 각종 error을 정의 해놈
from selenium.common.exceptions import ElementNotVisibleException
# driver timeout은 최대 시간 frequenct 얼마마다 확인 그동안 에러무시
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import time
import requests
from selenium import webdriver

def download(method,url,param=None, data=None, timeout=1, maxretries =3 ): 
    '''
    400이 나면 none
    500이 나면 다시시도
    '''
    try:
        # 이건 여기서 error 가 남
        # urlib에서는 객체를 만들고 urlopen으로 전달하는데 여기서는 한번에 실행
        resp=requests.request(method, url, params= param, data=data, headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"})
        
        # 테스트해보기 위해서 error를 발생시킴
        # error 발생안하면 넘어가는 명령어임
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        # 이경우는 서버상 error 즉 재시도를 요청해야함 500~600
        if 500<= e.response.status_code < 600 and maxretries > 0 and maxretries > 0:
            # 두가지 정의 필요 얼만큼 얼마 간격으로 재시도 할것인지
            # 몇초 기달릴지
            time.sleep(timeout)
            #얼만큼 재시도할지
            resp=download(method,url,param, data, timeout, maxretries-1)
            print("재시도")
        
        else:    
            print(e.response.status_code)
            print(e.response.reason)
    
    return resp

In [46]:
url = "https://www.jongno.go.kr/portal/bbs/selectBoardList.do"
data = {
    'bbsId':'BBSMSTR_000000000491',
    'menuNo':1123,
    'menuId':1123,
    'pageIndex':2
}

html = download('post',url, data=data)
dom = BeautifulSoup(html.text,"html.parser")
ID=[re.findall("[0-9]+",_["href"])[0] for _ in dom.select("table.list_type01 tbody tr td.output.tal.sj a")]
ID

['245106',
 '244972',
 '244749',
 '243745',
 '243017',
 '242658',
 '240788',
 '240178',
 '239868',
 '233059',
 '214871',
 '214870',
 '213641',
 '213531',
 '212801',
 '210993',
 '210827',
 '209943',
 '209764',
 '209762']

In [37]:
re.findall("[0-9]+","javascript:viewMove('245106')")[0]

'245106'

In [81]:
url = "https://www.jongno.go.kr/portal/bbs/selectBoardArticle.do"
data = {
    
    'bbsId':'BBSMSTR_000000000491',
    'menuNo':1123,
    'menuId':1123,
    'nttId':245106
}

html = download('post',url, data=data)
dom = BeautifulSoup(html.text,"html.parser")
dom.select("td.output")

[<td class="output" colspan="3">창신역에서 숭인교회올라가는 긴계단 청소</td>, <td class="output">
 				
 				2015년 05월 20일			
 			</td>, <td class="output">2908</td>, <td class="output" colspan="3">
 </td>, <td class="output" colspan="3"> $스마트폰으로 신고한 내용 다시 씁니다. 창신역에서 숭인교회 올라가는 138개 계단청소를 전부터 여러번 민원을 올렸으나 그때 잠깐 청소했을 뿐 항상 너무 더럽습니다. 어제는 보다못해 제가 종량제 쓰레기봉투 20L 짜리를 가지고 한번 수거했는데 (사진 참조) 금방 다 차서 다 못했습니다. 그리고 밤에는 가로등이 자꾸 꺼져 컴컴해서 무섭고 계단이 잘 안보여 위험하기조차 합니다 <br/>.1) 무단투기 경고 안내판을 붙이고<br/> 2) CCTV를 설치하고<br/> 3)가로등을 고쳐주시고<br/> 4)최소한 일주일에 한번이라도 정기적으로 청소해주셨으면 합니다 (강력요청)
 			</td>, <td class="output">도로과</td>, <td class="output">박영태</td>, <td class="output">3106</td>, <td class="output">pyt1021@mail.jongno.go.kr</td>, <td class="output" colspan="3">1. 구정 발전을 위하여 협조하여 주시는 선생님께 감사드립니다.<br/>         2. 먼저, 선생님께서 보안등 고장에 따라 불편을 겪으시게 된 것에 대해 죄송한 마음과 함께 사<br/>과의말씀을 드립니다. <br/>         3. 선생님께서 말씀하신 보안등은 2015. 5. 19. 현장 확인 후 보수하였음을 알려드립니다.<br/>         4. 기타 궁금하신 사항은 도로과(담당 박영태, 02-2148-3194)로 연락주시면 성심껏 답변드리<br/>겠습니다. 다시한번 선생님께 

In [85]:
#title = dom.select("td.output")[0].text
dom.select("td.output")[1].text
#content = dom.select("td.output")[4].text
#ans = dom.select("td.output")[-1].text

'\n\t\t\t\t\n\t\t\t\t2015년 05월 20일\t\t\t\n\t\t\t'

In [66]:
def get_id():
    result = []
    for i in range(1,4):
        url = "https://www.jongno.go.kr/portal/bbs/selectBoardList.do"
        data = {
            'bbsId':'BBSMSTR_000000000491',
            'menuNo':1123,
            'menuId':1123,
            'pageIndex':i
        }

        html = download('post',url, data=data)
        dom = BeautifulSoup(html.text,"html.parser")
        ID=[re.findall("[0-9]+",_["href"])[0] for _ in dom.select("table.list_type01 tbody tr td.output.tal.sj a")]
        result.extend(ID)
    return result

In [86]:
def get_data(ID):
    url = "https://www.jongno.go.kr/portal/bbs/selectBoardArticle.do"
    result = []
    for i in ID:
        data = {

            'bbsId':'BBSMSTR_000000000491',
            'menuNo':1123,
            'menuId':1123,
            'nttId':i
        }

        html = download('post',url, data=data)
        dom = BeautifulSoup(html.text,"html.parser")
        title = dom.select("td.output")[0].text
        date = dom.select("td.output")[1].text
        content = dom.select("td.output")[4].text
        ans = dom.select("td.output")[-1].text
        temp = [title,date,content,ans]
        result.append(temp)
    return result


In [87]:
ID = get_id()
a = get_data(ID)


In [88]:
a[11][1]

'\n\t\t\t\t\n\t\t\t\t2016년 07월 29일\t\t\t\n\t\t\t'